## 基于广告集数据用lora方法微调chatglm6B

### 1. 下载chatglm6B，进行微调前的推理体验

In [8]:
#测试未微调之前的效果
# !python inference_hf.py ../autodl-tmp/model/chatglm3 --prompt "类型#裙*版型#显瘦*材质#网纱*风格#性感*裙型#百褶*裙下摆#压褶*裙长#连衣裙*裙衣门襟#拉链*裙衣门襟#套头*裙款式#拼接*裙款式#拉链*裙款式#木耳边*裙款式#抽褶*裙款式#不规则"

from transformers import AutoTokenizer, AutoModel

path ="../autodl-tmp/model/chatglm3" #THUDM/chatglm3-6b

tokenizer = AutoTokenizer.from_pretrained(path, trust_remote_code=True)
model = AutoModel.from_pretrained(path, trust_remote_code=True, device='cuda')
model = model.eval()
response, history = model.chat(tokenizer, "类型#裙*版型#显瘦*材质#网纱*风格#性感*裙型#百褶*裙下摆#压褶*裙长#连衣裙*裙衣门襟#拉链*裙衣门襟#套头*裙款式#拼接*裙款式#拉链*裙款式#木耳边*裙款式#抽褶*裙款式#不规则", history=[])
print(response)

# response, history = model.chat(tokenizer, "晚上睡不着应该怎么办", history=history)
# print(response)



Setting eos_token is not supported, use the default one.
Setting pad_token is not supported, use the default one.
Setting unk_token is not supported, use the default one.


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

这是一款性感的百褶网纱连衣裙，裙下摆采用压褶设计，显得更加立体和修身。整体风格为性感，适合各种场合穿着。

裙衣门襟为拉链设计，方便穿脱。套头裙款式，无需脱卸脱卸衣衫，方便快捷。拼接和拉链的设计为裙款式增添了时尚元素。木耳边的设计和抽褶、不规则的裙款式相互呼应，让这款连衣裙更加有层次感。

这款连衣裙的材质为网纱，轻盈透气，带有一丝性感的气息。裙长适中，既能展现高挑的身材，又不失优雅。整体设计为百褶裙型，显瘦效果显著，让你在穿着过程中更加自信。


### 2. 准备数据集

In [1]:
import json
from typing import Union
from pathlib import Path


def _resolve_path(path: Union[str, Path]) -> Path:
    return Path(path).expanduser().resolve()


def _mkdir(dir_name: Union[str, Path]):
    dir_name = _resolve_path(dir_name)
    if not dir_name.is_dir():
        dir_name.mkdir(parents=True, exist_ok=False)


def convert_adgen(data_dir: Union[str, Path], save_dir: Union[str, Path]):
    def _convert(in_file: Path, out_file: Path):
        _mkdir(out_file.parent)
        with open(in_file, encoding='utf-8') as fin:
            # 使用 with 语句打开输出文件 out_file，并将其赋值给变量 fout。指定了以文本模式写入（'wt'）和 UTF-8 编码打开文件。
            with open(out_file, 'wt', encoding='utf-8') as fout:
                for line in fin:
                    # 将读取的每行内容解析为 JSON 格式，存储在变量 dct 中。
                    dct = json.loads(line)
                    # 根据解析得到的 JSON 数据构造一个新的数据结构 sample，其中包含一个键为 'conversations' 的列表，列表中包含两个字典，分别表示用户和助手的对话内容。
                    sample = {'conversations': [{'role': 'user', 'content': dct['content']},
                                                {'role': 'assistant', 'content': dct['summary']}]}
                    # 将构造好的 sample 对象转换为 JSON 字符串，并写入到输出文件 fout 中。
                    # ensure_ascii=False 表示输出的 JSON 字符串中允许包含非 ASCII 字符，+ '\n' 则是为了在每行结尾添加换行符，使输出文件每行一条数据。
                    fout.write(json.dumps(sample, ensure_ascii=False) + '\n')
    
    # 如果 data_dir 是相对路径 ./data，经过 _resolve_path 处理后，可能会得到 /path/to/current/directory/data 这样的绝对路径。
    # 这样做的好处是，无论当前工作目录在哪里，都可以准确地找到数据和保存目录，避免了路径混乱或不确定性带来的问题。
    data_dir = _resolve_path(data_dir)
    save_dir = _resolve_path(save_dir)

    train_file = data_dir / 'train.json'
    if train_file.is_file():
        out_file = save_dir / train_file.relative_to(data_dir)
        _convert(train_file, out_file)

    dev_file = data_dir / 'dev.json'
    if dev_file.is_file():
        out_file = save_dir / dev_file.relative_to(data_dir)
        _convert(dev_file, out_file)


convert_adgen('data/AdvertiseGen', 'data/AdvertiseGen_fix')

### 3.微调模型

In [4]:
!python finetune_hf.py  data/AdvertiseGen_fix  ../autodl-tmp/model/chatglm3  lora.yaml

# 从保存点进行微调
# 如果按照上述方式进行训练，每次微调都会从头开始，如果你想从训练一半的模型开始微调，你可以加入第四个参数，这个参数有两种传入方式:

# yes, 自动从最后一个保存的 Checkpoint开始训练
# XX, 断点号数字 例 600 则从序号600 Checkpoint开始训练
# 例如，这就是一个从最后一个保存点继续微调的示例代码

# cd finetune_demo
# python finetune_hf.py  data/AdvertiseGen/  THUDM/chatglm3-6b  configs/lora.yaml yes

Setting eos_token is not supported, use the default one.
Setting pad_token is not supported, use the default one.
Setting unk_token is not supported, use the default one.
Loading checkpoint shards: 100%|██████████████████| 7/7 [00:01<00:00,  3.99it/s]
trainable params: 1,949,696 || all params: 6,245,533,696 || trainable%: 0.031217444255383614
--> Model

--> model has 1.949696M params

Setting num_proc from 16 back to 1 for the train split to disable multiprocessing as it only contains one shard.
Generating train split: 114599 examples [00:00, 551704.99 examples/s]
Setting num_proc from 16 back to 1 for the validation split to disable multiprocessing as it only contains one shard.
Generating validation split: 1070 examples [00:00, 129122.34 examples/s]
Setting num_proc from 16 back to 1 for the test split to disable multiprocessing as it only contains one shard.
Generating test split: 1070 examples [00:00, 125423.54 examples/s]
原始列名：['conversations']
Map (num_proc=16): 100%|██████| 1145

In [42]:
#加入早停逻辑，在上面的微调进展下继续微调，并在finetune_hf.py加入限制条件，以让loss结果回归到3.25以下
# 更准确的说是到达3.25以下则停止训练

!python finetune_hf.py  data/AdvertiseGen_fix  ../autodl-tmp/model/chatglm3  lora.yaml yes

Setting eos_token is not supported, use the default one.
Setting pad_token is not supported, use the default one.
Setting unk_token is not supported, use the default one.
Loading checkpoint shards: 100%|██████████████████| 7/7 [00:01<00:00,  3.95it/s]
trainable params: 1,949,696 || all params: 6,245,533,696 || trainable%: 0.031217444255383614
--> Model

--> model has 1.949696M params

原始列名：['conversations']
train_dataset: Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 114599
})
原始列名：['conversations']
val_dataset: Dataset({
    features: ['input_ids', 'output_ids'],
    num_rows: 1070
})
原始列名：['conversations']
test_dataset: Dataset({
    features: ['input_ids', 'output_ids'],
    num_rows: 1070
})
--> Sanity check
           '[gMASK]': 64790 -> -100
               'sop': 64792 -> -100
          '<|user|>': 64795 -> -100
                  '': 30910 -> -100
                '\n': 13 -> -100
                  '': 30910 -> -100
                '类型': 33467 -> -100
            

### 4. 使用微调的数据集进行推理

In [43]:
# !python inference_hf.py output/checkpoint-2000/ --prompt "类型#裙*版型#显瘦*材质#网纱*风格#性感*裙型#百褶*裙下摆#压褶*裙长#连衣裙*裙衣门襟#拉链*裙衣门襟#套头*裙款式#拼接*裙款式#拉链*裙款式#木耳边*裙款式#抽褶*裙款式#不规则"

!python inference_hf.py output_model/ --prompt "类型#裙*版型#显瘦*材质#网纱*风格#性感*裙型#百褶*裙下摆#压褶*裙长#连衣裙*裙衣门襟#拉链*裙衣门襟#套头*裙款式#拼接*裙款式#拉链*裙款式#木耳边*裙款式#抽褶*裙款式#不规则"

Loading checkpoint shards: 100%|██████████████████| 7/7 [00:03<00:00,  2.19it/s]
Setting eos_token is not supported, use the default one.
Setting pad_token is not supported, use the default one.
Setting unk_token is not supported, use the default one.
prompt=类型#裙*版型#显瘦*材质#网纱*风格#性感*裙型#百褶*裙下摆#压褶*裙长#连衣裙*裙衣门襟#拉链*裙衣门襟#套头*裙款式#拼接*裙款式#拉链*裙款式#木耳边*裙款式#抽褶*裙款式#不规则
等待回复...
这款连衣裙的版型很显瘦，压褶的木耳边的设计，很甜美，加上网纱拼接，很性感。网纱的百褶裙摆，很飘逸。套头拉链的设计，很方便，拉链的压褶设计，很别致，很时尚。


### 5. 将微调后的模型部署到 basic_demo下的 gradio_demo 中，并能够通过 webui 来进行调用。

In [8]:
# web_demo_gradio.py已调整好
!python web_demo_gradio.py

Loading checkpoint shards: 100%|██████████████████| 7/7 [00:03<00:00,  2.24it/s]
Setting eos_token is not supported, use the default one.
Setting pad_token is not supported, use the default one.
Setting unk_token is not supported, use the default one.
Running on local URL:  http://127.0.0.1:6006

To create a public link, set `share=True` in `launch()`.
^C
Keyboard interruption in main thread... closing server.
